In [13]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

# Set paths
image_dir = 'C:/Users/Rahila/Desktop/project/data/images'
label_dir = 'C:/Users/Rahila/Desktop/project/data/labels/'

# Define YOLO class labels
class_labels = [
    "camouflage_soldier", "weapon", "military_tank", "military_truck",
    "military_vehicle", "civilian", "soldier", "civilian_vehicle", "trench"
]

# Function to parse YOLO-format label file
def parse_annotation(label_file):
    with open(label_file, 'r') as f:
        lines = f.readlines()
        parsed = []
        for line in lines:
            parts = line.strip().split()
            if len(parts) == 5:  # Expecting class_id x_center y_center width height
                parsed.append(parts)
        return parsed

# Collect annotations from label files
ann_stats = []
for label_file in os.listdir(label_dir):
    if not label_file.endswith('.txt'):
        continue
    full_path = os.path.join(label_dir, label_file)
    objects = parse_annotation(full_path)
    for obj in objects:
        try:
            class_id, x, y, w, h = map(float, obj)
            ann_stats.append({
                "file": label_file.replace('.txt', ''),
                "class_id": int(class_id),
                "x_center": x,
                "y_center": y,
                "width": w,
                "height": h
            })
        except ValueError:
            print(f"Skipping bad line in {label_file}: {obj}")

# Check if any annotations were found
if not ann_stats:
    raise ValueError("No annotation data found. Please check the label_dir path and files.")

# Create DataFrame from parsed annotations
df = pd.DataFrame(ann_stats)

# Map class_id to class names
df["class_name"] = df["class_id"].apply(lambda x: class_labels[x] if x < len(class_labels) else "unknown")

# Plot class distribution
plt.figure(figsize=(10, 6))
sns.countplot(y='class_name', data=df, order=df['class_name'].value_counts().index)
plt.title("Class Distribution of Labeled Objects")
plt.xlabel("Number of Instances")
plt.ylabel("Class")
plt.tight_layout()
plt.show()

# (Optional) Show average object width and height by class
plt.figure(figsize=(10, 6))
sns.barplot(x='class_name', y='width', data=df, estimator='mean', ci=None)
plt.xticks(rotation=45)
plt.title("Average Object Width by Class")
plt.ylabel("Width (normalized)")
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 6))
sns.barplot(x='class_name', y='height', data=df, estimator='mean', ci=None)
plt.xticks(rotation=45)
plt.title("Average Object Height by Class")
plt.ylabel("Height (normalized)")
plt.tight_layout()
plt.show()


ValueError: No annotation data found. Please check the label_dir path and files.

In [ ]:
from ultralytics import YOLO

# Create YOLO model (v8n = nano, v8s = small, v8m = medium)
model = YOLO('yolov8s.pt')

# Train on custom dataset
model.train(data='data.yaml', epochs=50, imgsz=640, batch=16)

# Save model
model.export(format='onnx')


In [ ]:
from ultralytics import YOLO
import cv2

model = YOLO('runs/detect/train/weights/best.pt')  # Your trained model

cap = cv2.VideoCapture(0)  # Use 0 for webcam

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)[0]
    annotated = results.plot()
    cv2.imshow("Military Object Detection", annotated)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


In [ ]:
from ultralytics import YOLO

model = YOLO('runs/detect/train/weights/best.pt')
metrics = model.val()
print("Precision:", metrics.box.map50)
print("Recall:", metrics.box.recall)
print("mAP:", metrics.box.map)
